In [ ]:
# Mount Google Drive to access your dataset
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


In [ ]:
# Parameters
SEQUENCE_LENGTH = 10
IMG_SIZE = (224, 224)
BATCH_SIZE = 8
EPOCHS = 10
DATASET_PATH = '/content/drive/MyDrive/action_data'  # Change to your folder path in Drive


In [ ]:
# Function to load video frames
def load_video_frames(video_path, sequence_length=SEQUENCE_LENGTH, frame_size=IMG_SIZE):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while count < sequence_length:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
        count += 1
    cap.release()
    # Pad with black frames if less
    if len(frames) < sequence_length:
        padding = [np.zeros((frame_size[0], frame_size[1], 3), dtype=np.uint8)] * (sequence_length - len(frames))
        frames.extend(padding)
    frames = np.array(frames)
    return frames

# Load dataset from folders and assign labels
def load_dataset(data_dir, classes=['punch', 'non_punch']):
    X = []
    y = []
    for label, cls in enumerate(classes):
        cls_path = os.path.join(data_dir, cls)
        for file in os.listdir(cls_path):
            if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                video_path = os.path.join(cls_path, file)
                frames = load_video_frames(video_path)
                X.append(frames)
                y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Load all data
X, y = load_dataset(DATASET_PATH)

# Split data: 70% train, 15% val, 15% test with stratification for classes
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(f"Train samples: {len(X_train)}, Val samples: {len(X_val)}, Test samples: {len(X_test)}")


Train samples: 228, Val samples: 49, Test samples: 50


In [ ]:
# Preprocess frames with MobileNetV2 preprocessing function
def preprocess_sequences(sequences):
    preprocessed = np.empty_like(sequences, dtype=np.float32)
    for i, seq in enumerate(sequences):
        preprocessed[i] = preprocess_input(seq.astype(np.float32))
    return preprocessed

X_train = preprocess_sequences(X_train)
X_val = preprocess_sequences(X_val)
X_test = preprocess_sequences(X_test)

In [ ]:
# Build model with TimeDistributed MobileNetV2 + LSTM
input_shape = (SEQUENCE_LENGTH, IMG_SIZE[0], IMG_SIZE[1], 3)
inputs = Input(shape=input_shape)

base_cnn = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_cnn.trainable = False

x = TimeDistributed(base_cnn)(inputs)
x = TimeDistributed(GlobalAveragePooling2D())(x)
x = LSTM(64, return_sequences=True)(x)
x = LSTM(128)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(len(np.unique(y)), activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 10, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 10, 7, 7, 1280) │     2,257,984 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 10, 1280)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 10, 64)         │       344,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,709,506 (10.34 MB)

 Trainable params: 451,522 (1.72 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Train model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE)


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 64s 717ms/step - accuracy: 0.8100 - loss: 0.4015 - val_accuracy: 1.0000 - val_loss: 0.0346
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 186ms/step - accuracy: 1.0000 - loss: 0.0157 - val_accuracy: 1.0000 - val_loss: 0.0052
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 203ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 1.0000 - val_loss: 9.0595e-04
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 212ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 1.0000 - val_loss: 2.3121e-04
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 199ms/step - accuracy: 1.0000 - loss: 4.3320e-04 - val_accuracy: 1.0000 - val_loss: 3.5569e-04
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 188ms/step - accuracy: 1.0000 - loss: 2.4376e-04 - val_accuracy: 1.0000 - val_loss: 2.8931e-04
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 207ms/step - accuracy: 1.0000 - loss: 1.9510e-04 - val_accuracy: 1.0000 - val_loss: 2.0532e-04
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 198ms/step - accuracy: 1

In [ ]:
# Evaluate model on test set
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision (macro): {precision_score(y_test, y_pred, average='macro'):.4f}")
print(f"Recall (macro): {recall_score(y_test, y_pred, average='macro'):.4f}")
print(f"F1 Score (macro): {f1_score(y_test, y_pred, average='macro'):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['punch', 'non_punch']))

2/2 ━━━━━━━━━━━━━━━━━━━━ 24s 11s/step
Test Accuracy: 0.9800
Precision (macro): 0.9833
Recall (macro): 0.9762
F1 Score (macro): 0.9793
Confusion Matrix:
 [[20  1]
 [ 0 29]]
Classification Report:
               precision    recall  f1-score   support

       punch       1.00      0.95      0.98        21
   non_punch       0.97      1.00      0.98        29

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

